In [2]:
import torch
import ssl
import matplotlib.pyplot as plt
from src.logger.logger_initializer import LoggerInitializer

ssl._create_default_https_context = ssl._create_unverified_context
device = 'cuda'
torch.__version__

LoggerInitializer().init()

True

In [3]:
from src.data_loader.dataset_getter import DatasetGetter
dataset = DatasetGetter('../../bayesian_test/data', batch_size=12).get()

In [4]:
from src.models.cnn.resnet50_cnn_classifier import Resnet50CnnClassifier
classifier = Resnet50CnnClassifier(dataset=dataset, device=device).init()

In [ ]:
classifier.freeze_all_layers_except_fc()
result_df, result_dict = classifier.fit(num_epoch=50)
display(result_df)
print(result_dict)

[2023-04-17 02:05:04] 21832 src.scoring.epoch_score_printer {epoch_score_printer-21} INFO - [Epoch 0/50]: f1_score: 0.732 accuracy_score: 0.754 loss: 102.373 
[2023-04-17 02:05:11] 21832 src.scoring.epoch_score_printer {epoch_score_printer-21} INFO - [Epoch 1/50]: f1_score: 0.709 accuracy_score: 0.727 loss: 87.317 
[2023-04-17 02:05:19] 21832 src.scoring.epoch_score_printer {epoch_score_printer-21} INFO - [Epoch 2/50]: f1_score: 0.707 accuracy_score: 0.723 loss: 76.492 
[2023-04-17 02:05:26] 21832 src.scoring.epoch_score_printer {epoch_score_printer-21} INFO - [Epoch 3/50]: f1_score: 0.725 accuracy_score: 0.750 loss: 76.295 
[2023-04-17 02:05:34] 21832 src.scoring.epoch_score_printer {epoch_score_printer-21} INFO - [Epoch 4/50]: f1_score: 0.673 accuracy_score: 0.691 loss: 70.267 
[2023-04-17 02:05:41] 21832 src.scoring.epoch_score_printer {epoch_score_printer-21} INFO - [Epoch 5/50]: f1_score: 0.711 accuracy_score: 0.742 loss: 69.019 
[2023-04-17 02:05:48] 21832 src.scoring.epoch_score

In [ ]:
classifier.freeze_all_layers()
train_all_result_df, train_all_result_dict = classifier.fit(num_epoch=200)

In [6]:
classifier.save_model(result_dict)

In [7]:
test_model_weights_path = '../../bayesian_refactoring/assets/model_weights/resnet50cnn_5912_813dec81-d9db-43f9-8e68-a4792822a394.pt'

In [8]:
test_results = torch.load(test_model_weights_path)

In [10]:
list(test_results.keys())

['best_loss',
 'best_f1_score',
 'best_accuracy_score',
 'best_loss_model',
 'best_f1_score_model',
 'best_accuracy_score_model']

In [9]:
new_classifier = Resnet50CnnClassifier(dataset=dataset, device=device).load_model(model_state_dict=test_results['best_f1_score_model'])

In [10]:
y_true, y_pred = new_classifier.predict(dataset.validate_dataloader)

In [11]:
new_classifier.score(dataset.validate_dataloader)

{'f1_score': 0.7022692266004554, 'accuracy_score': 0.7351778656126482}